In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Análisis exploratorio + Pre-procesamiento - Dataset Sevilla

In [ ]:
# Variable que dictamina si la nomenclatura de las imagenes debe ser tratada 
PROCESAR_IMAGENES_NAME = False 

In [ ]:
if PROCESAR_IMAGENES_NAME == True:
    # Carpeta principal que contiene las subcarpetas con las fotos
    carpeta_principal = "./Sevilla_Data/ENTRENAMIENTO HUVR/"
    contador = 1    
    # Recorremos recursivamente todas las subcarpetas
    for ruta_actual, carpetas, archivos in os.walk(carpeta_principal):
        # Iteramos sobre cada archivo en la carpeta actual
        for archivo in archivos:
            # Creamos el nuevo nombre del archivo
            nuevo_nombre = f"Sevilla_{contador}.jpg" 
            
            # Ruta completa del archivo antiguo y nuevo
            ruta_antigua = os.path.join(ruta_actual, archivo)
            ruta_nueva = os.path.join(ruta_actual, nuevo_nombre)
            
            os.rename(ruta_antigua, ruta_nueva)
            contador += 1

In [ ]:
# Función para obtener el nombre de los archivos y sus categorías
def obtener_archivos_y_categorias(ruta):
    archivos_y_categorias = []
    for categoria in os.listdir(ruta):
        categoria_ruta = os.path.join(ruta, categoria)
        if os.path.isdir(categoria_ruta):
            for archivo in os.listdir(categoria_ruta):
                archivo_ruta = os.path.join(categoria_ruta, archivo)
                if os.path.isfile(archivo_ruta):
                    archivos_y_categorias.append((archivo.rstrip('.jpg'), categoria))
    return archivos_y_categorias

In [ ]:
# Ruta de la carpeta que contiene las categorías
ruta_carpeta = "D:\TFM\Codigo\Sevilla_Data\ENTRENAMIENTO HUVR"

archivos_y_categorias = obtener_archivos_y_categorias(ruta_carpeta)

# Creamos un DataFrame con los datos
data = pd.DataFrame(archivos_y_categorias, columns=['id', 'diagnosis'])
print(data)

In [ ]:
# Configuramos estilo y paleta de colores de Seaborn
sns.set(style="whitegrid", palette="pastel")

count_data = data['diagnosis'].value_counts().reset_index()
count_data.columns = ['diagnosis', 'Count']
ax = sns.barplot(data=count_data, x='Count', y='diagnosis', hue='diagnosis', dodge=False, legend=False)

# Añade etiquetas y título
plt.xlabel('Muestras')
plt.ylabel('Diagnóstico')
plt.title('Muestras según el tipo de diagnóstico')
plt.show()

In [ ]:
# Definimos una función para asignar los valores benigno/maligno según los valores de la columna categórica
def asignar_etiqueta(valor):
    if valor in ['melanoma', 'carcinoma']:
        return '1'
    else:
        return '0'

In [ ]:
data['benign_malignant'] = data['diagnosis'].apply(asignar_etiqueta)

In [ ]:
data

In [ ]:
count_data = data['benign_malignant'].value_counts().reset_index()
count_data.columns = ['Diagnosis', 'Count']

# Count de casos benignos y malignos
num_benign = count_data[count_data['Diagnosis'] == '0']['Count'].values[0]
num_malignant = count_data[count_data['Diagnosis'] == '1']['Count'].values[0]

ax = sns.barplot(data=count_data, x='Diagnosis', y='Count', hue='Diagnosis', dodge=False, legend=False)
plt.xlabel('Tipo')
plt.ylabel('Count')
plt.title('Ratio de benigno versus maligno\nBenignos: {} | Malignos: {}'.format(num_benign, num_malignant))
plt.show()

In [ ]:
# Añadimos los factores climatoloógicos obtenidos anteriormente al conjunto

#Temperatura
data['max_temp'] = 42.6
data['min_tmp'] = 0.8
data['mean_temp'] = 19.064186438636636
data['median_temp'] = 18
data['std_temp'] = 7.413991121799422

#Dew Point
data['max_dew_point'] = 23.5
data['min_dew_point'] = -5.7
data['mean_dew_point'] = 11.278210238940833
data['median_dew_point'] = 12.0
data['std_dew_point'] = 4.186711966042453

#Wind Speed
data['max_wind_speed'] = 55.44
data['min_wind_speed'] = 0.0
data['mean_wind_speed'] = 9.833168251811594
data['median_wind_speed'] = 9.26
data['std_wind_speed'] = 6.915781571727125

#Wind Direction
data['max_wind_direction'] = 360.0
data['min_wind_direction'] = 10.0
data['mean_wind_direction'] = 169.82415192425006
data['median_wind_direction'] = 200.0
data['std_wind_direction'] = 95.9660244878841

#Visibility
data['max_visibility'] = 30.0
data['min_visibility'] = 0.0
data['mean_visibility'] = 10.770878488037276
data['median_visibility'] = 10.0
data['std_visibility'] = 4.90348844228675

In [ ]:
nan_check = data.isna().any().any()

if nan_check:
    print("Hay valores NaN en el DataFrame.")
else:
    print("No hay valores NaN en el DataFrame.")

In [ ]:
# Leemos el nuevo dataset de calculos solares
data_calcs = pd.read_csv('./Sevilla_Data/Sevilla_solar_calcs.csv')
data_calcs

In [ ]:
# Transformamos el nuevo dataset para obtener una estructura entendible y facilmente iterable 
variables = [
    "ALLSKY_KT",
    "CLOUD_AMT",
    "TOA_SW_DWN",
    "ALLSKY_SFC_UVA",
    "ALLSKY_SFC_UVB",
    "ALLSKY_SRF_ALB",
    "ALLSKY_SFC_SW_DNI",
    "ALLSKY_SFC_SW_DWN",
    "ALLSKY_SFC_PAR_TOT",
    "ALLSKY_SFC_SW_DIFF"
]

columns = [
    "max",
    "min",
    "mean",
    "median",
    "std",
]

data_calcs.index = variables
data_calcs.columns = columns

data_calcs

In [ ]:
# Introducimos los nuevos datos al dataset original
for index, row in data_calcs.iterrows():
    nombre = row.name  
    for columna in data_calcs.columns:
        estadistica = columna  
        nuevo_nombre_columna = f"{nombre}_{estadistica}"  
        data[nuevo_nombre_columna] = row[columna]

In [ ]:
data

In [ ]:
#Guardamos el dataset de factores externos
data.to_csv('./Sevilla_Data/final_metadata_Sevilla.csv', index=False)